In [16]:
import sqlite3
import numpy as np
import pandas as pd

#import the required packages 

In [41]:
conn = sqlite3.connect('/Users/tanishalohchab/Documents/SQL Database/hmsdatabase.sqlite')
cursor = conn.cursor()

In [18]:
# Create Patient table
cursor.execute('''
CREATE TABLE Patient (
    patientid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    address TEXT NOT NULL,
    phone TEXT NOT NULL,
    insuranceid TEXT NOT NULL,
    pcp INTEGER NOT NULL,
    FOREIGN KEY (pcp) REFERENCES Physician(physicianid)
)
''')

# Create Procedure table
cursor.execute('''
CREATE TABLE Procedure (
    procedureid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    cost REAL NOT NULL
)
''')

# Create Room table
cursor.execute('''
CREATE TABLE Room (
    roomnumber INTEGER PRIMARY KEY,
    roomtype TEXT NOT NULL,
    unavailable BOOLEAN NOT NULL
)
''')

# Create Physician table
cursor.execute('''
CREATE TABLE Physician (
    physicianid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    position TEXT NOT NULL
)
''')

# Create Undergoes table
cursor.execute('''
CREATE TABLE Undergoes (
    patientid INTEGER NOT NULL,
    procedureid INTEGER NOT NULL,
    stayid INTEGER NOT NULL,
    date TEXT NOT NULL,
    PRIMARY KEY (patientid, procedureid, stayid),
    FOREIGN KEY (patientid) REFERENCES Patient(patientid),
    FOREIGN KEY (procedureid) REFERENCES Procedure(procedureid),
    FOREIGN KEY (stayid) REFERENCES Stay(stayid)
)
''')

# Create PhysicianProcedure table
cursor.execute('''
CREATE TABLE PhysicianProcedure (
    physicianid INTEGER NOT NULL,
    procedureid INTEGER NOT NULL,
    PRIMARY KEY (physicianid, procedureid),
    FOREIGN KEY (physicianid) REFERENCES Physician(physicianid),
    FOREIGN KEY (procedureid) REFERENCES Procedure(procedureid)
)
''')

# Create Stay table
cursor.execute('''
CREATE TABLE Stay (
    stayid INTEGER PRIMARY KEY,
    patientid INTEGER NOT NULL,
    start_time TEXT NOT NULL,
    end_time TEXT NOT NULL,
    roomnumber INTEGER NOT NULL,
    FOREIGN KEY (patientid) REFERENCES Patient(patientid),
    FOREIGN KEY (roomnumber) REFERENCES Room(roomnumber)
)
''')

# Commit the transaction
conn.commit()


In [19]:
cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
print('List of Tables present in the Database')
table_list = [table[0] for table in cursor.fetchall()]
table_list

List of Tables present in the Database


['Patient',
 'Procedure',
 'Room',
 'Physician',
 'Undergoes',
 'PhysicianProcedure',
 'Stay']

In [20]:

# Insert data into Patient table
patients = [
    (100000001, 'John Smith', '42 Foobar Lane', '555-0256', '68476213', 1),
    (100000002, 'Grace Ritchie', '37 Snafu Drive', '555-0512', '36546321', 2),
    (100000003, 'Random J. Patient', '101 Omgbbq Street', '555-1204', '65465421', 2),
    (100000004, 'Dennis Doe', '1100 Foobaz Avenue', '555-2048', '68421879', 3),
]
cursor.executemany('INSERT INTO Patient VALUES (?,?,?,?,?,?)', patients)

# Insert data into Procedure table
procedures = [
    (1, 'Reverse Rhinopodoplasty', 1500.00),
    (2, 'Obtuse Pyloric Recombobulation', 3750.00),
    (3, 'Folded Demiophtalmectomy', 4500.00),
    (4, 'Complete Walletectomy', 10000.00),
    (5, 'Obfuscated Dermogastrotomy', 4899.00),
    (6, 'Reversible Pancreomyoplasty', 5600.00),
    (7, 'Follicular Demiectomy', 25.00),
]
cursor.executemany('INSERT INTO Procedure VALUES (?,?,?)', procedures)

# Insert data into Room table
rooms = [
    (101, 'Single', 0),
    (102, 'Single', 1),
    (103, 'Double', 0),
    (111, 'Single', 0),
    (112, 'Single', 1),
    (113, 'Double', 0),
    (121, 'Single', 0),
    (122, 'Single', 0),
    (123, 'Single', 0),
]
cursor.executemany('INSERT INTO Room VALUES (?,?,?)', rooms)

# Insert data into Physician table
physicians = [
    (1, 'John Dorian', 'Staff Internist'),
    (2, 'Elliot Reid', 'Attending Physician'),
    (3, 'Christopher Turk', 'Surgical Attending Physician'),
    (4, 'Percival Cox', 'Senior Attending Physician'),
    (5, 'Bob Kelso', 'Head Chief of Medicine'),
    (6, 'Todd Quinlan', 'Surgical Attending Physician'),
    (7, 'John Wen', 'Surgical Attending Physician'),
    (8, 'Keith Dudemeister', 'MD Resident'),
    (9, 'Molly Clock', 'Attending Psychiatrist'),
]
cursor.executemany('INSERT INTO Physician VALUES (?,?,?)', physicians)

# Insert data into Undergoes table
undergoes = [
    (100000001, 6, 3215, '2008-02-05'),
    (100000001, 2, 3215, '2008-03-05'),
    (100000004, 1, 3217, '2008-07-05'),
    (100000004, 5, 3217, '2008-09-05'),
    (100000001, 7, 3215, '2008-10-05'),
    (100000004, 4, 3217, '2008-13-05'),  # This date is invalid, assuming a typo
]
cursor.executemany('INSERT INTO Undergoes VALUES (?,?,?,?)', undergoes)

# Insert data into PhysicianProcedure table
physician_procedures = [
    (3, 6),
    (7, 2),
    (3, 1),
    (6, 5),
    (7, 7),
    (3, 4),
]
cursor.executemany('INSERT INTO PhysicianProcedure VALUES (?,?)', physician_procedures)

# Insert data into Stay table
stays = [
    (3215, 100000001, '2008-01-05', '2008-04-05', 111),
    (3216, 100000003, '2008-03-05', '2008-14-05', 123),  # This end date is invalid, assuming a typo
    (3217, 100000004, '2008-02-05', '2008-03-05', 112),
]
cursor.executemany('INSERT INTO Stay VALUES (?,?,?,?,?)', stays)

# Commit the transaction
conn.commit()

In [21]:
#sqlite_master is a table with database schema
pd.read_sql(""" SELECT *
                FROM sqlite_master
                WHERE type='table';""",
           conn)

,type,name,tbl_name,rootpage,sql
0,table,Patient,Patient,2,CREATE TABLE Patient (\n patientid INTEGER ...
1,table,Procedure,Procedure,3,CREATE TABLE Procedure (\n procedureid INTE...
2,table,Room,Room,4,CREATE TABLE Room (\n roomnumber INTEGER PR...
3,table,Physician,Physician,5,CREATE TABLE Physician (\n physicianid INTE...
4,table,Undergoes,Undergoes,6,CREATE TABLE Undergoes (\n patientid INTEGE...
5,table,PhysicianProcedure,PhysicianProcedure,8,CREATE TABLE PhysicianProcedure (\n physici...
6,table,Stay,Stay,10,CREATE TABLE Stay (\n stayid INTEGER PRIMAR...


In [23]:
pd.read_sql("""SELECT * FROM Patient ORDER BY patientid;""", conn)

,patientid,name,address,phone,insuranceid,pcp
0,100000001,John Smith,42 Foobar Lane,555-0256,68476213,1
1,100000002,Grace Ritchie,37 Snafu Drive,555-0512,36546321,2
2,100000003,Random J. Patient,101 Omgbbq Street,555-1204,65465421,2
3,100000004,Dennis Doe,1100 Foobaz Avenue,555-2048,68421879,3


In [24]:
pd.read_sql("""SELECT * FROM Procedure ORDER BY procedureid;""", conn)

,procedureid,name,cost
0,1,Reverse Rhinopodoplasty,1500.0
1,2,Obtuse Pyloric Recombobulation,3750.0
2,3,Folded Demiophtalmectomy,4500.0
3,4,Complete Walletectomy,10000.0
4,5,Obfuscated Dermogastrotomy,4899.0
5,6,Reversible Pancreomyoplasty,5600.0
6,7,Follicular Demiectomy,25.0


In [25]:
pd.read_sql("""SELECT * FROM Room ORDER BY roomnumber;""", conn)

,roomnumber,roomtype,unavailable
0,101,Single,0
1,102,Single,1
2,103,Double,0
3,111,Single,0
4,112,Single,1
5,113,Double,0
6,121,Single,0
7,122,Single,0
8,123,Single,0


In [26]:
pd.read_sql("""SELECT * FROM Physician ORDER BY physicianid;""", conn)

,physicianid,name,position
0,1,John Dorian,Staff Internist
1,2,Elliot Reid,Attending Physician
2,3,Christopher Turk,Surgical Attending Physician
3,4,Percival Cox,Senior Attending Physician
4,5,Bob Kelso,Head Chief of Medicine
5,6,Todd Quinlan,Surgical Attending Physician
6,7,John Wen,Surgical Attending Physician
7,8,Keith Dudemeister,MD Resident
8,9,Molly Clock,Attending Psychiatrist


In [27]:
pd.read_sql("""SELECT * FROM Undergoes ORDER BY patientid, procedureid, stayid;""", conn)

,patientid,procedureid,stayid,date
0,100000001,2,3215,2008-03-05
1,100000001,6,3215,2008-02-05
2,100000001,7,3215,2008-10-05
3,100000004,1,3217,2008-07-05
4,100000004,4,3217,2008-13-05
5,100000004,5,3217,2008-09-05


In [28]:
 pd.read_sql("""SELECT * FROM PhysicianProcedure ORDER BY physicianid, procedureid;""", conn)

,physicianid,procedureid
0,3,1
1,3,4
2,3,6
3,6,5
4,7,2
5,7,7


In [29]:
pd.read_sql("""SELECT * FROM Stay ORDER BY stayid;""", conn)

,stayid,patientid,start_time,end_time,roomnumber
0,3215,100000001,2008-01-05,2008-04-05,111
1,3216,100000003,2008-03-05,2008-14-05,123
2,3217,100000004,2008-02-05,2008-03-05,112


2. Write an SQL involving the junction table and two other related tables. You must use the INNER JOIN to connect with all three tables. The database that you created must be included in your SQL queries.


This query would be useful for generating a report of all procedures undergone by patients, including the details of each procedure and the corresponding patient information.

In [43]:




pd.read_sql("""SELECT 
    Patient.name AS PatientName, 
    Procedure.name AS ProcedureName, 
    Undergoes.date AS ProcedureDate
FROM 
    Undergoes
INNER JOIN 
    Patient ON Undergoes.patientid = Patient.patientid
INNER JOIN 
    Procedure ON Undergoes.procedureid = Procedure.procedureid
ORDER BY 
    Undergoes.date;
""", conn)

,PatientName,ProcedureName,ProcedureDate
0,John Smith,Reversible Pancreomyoplasty,2008-02-05
1,John Smith,Obtuse Pyloric Recombobulation,2008-03-05
2,Dennis Doe,Reverse Rhinopodoplasty,2008-07-05
3,Dennis Doe,Obfuscated Dermogastrotomy,2008-09-05
4,John Smith,Follicular Demiectomy,2008-10-05
5,Dennis Doe,Complete Walletectomy,2008-13-05


3. Write an SQL by including two or more tables and using the LEFT OUTER JOIN. Show the results and sort the results by key field(s). Interpret the results compared to what an INNER JOIN does.

In [31]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName, 
    Patient.patientid AS PatientID, 
    Physician.name AS PhysicianName,
    Physician.position AS PhysicianPosition
FROM 
    Patient
LEFT OUTER JOIN 
    Physician ON Patient.pcp = Physician.physicianid
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,PatientID,PhysicianName,PhysicianPosition
0,John Smith,100000001,John Dorian,Staff Internist
1,Grace Ritchie,100000002,Elliot Reid,Attending Physician
2,Random J. Patient,100000003,Elliot Reid,Attending Physician
3,Dennis Doe,100000004,Christopher Turk,Surgical Attending Physician


4. Write a single-row subquery. Show the results and sort the results by key field(s). Interpret the output.

In [32]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName,
    Undergoes.procedureid,
    Procedure.name AS ProcedureName,
    Procedure.cost
FROM 
    Patient
INNER JOIN 
    Undergoes ON Patient.patientid = Undergoes.patientid
INNER JOIN 
    Procedure ON Undergoes.procedureid = Procedure.procedureid
WHERE 
    Procedure.cost = (SELECT MAX(cost) FROM Procedure)
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,procedureid,ProcedureName,cost
0,Dennis Doe,4,Complete Walletectomy,10000.0


5. Write a multiple-row subquery. Show the results and sort the results by key field(s). Interpret the output.

In [33]:
pd.read_sql("""SELECT 
    Physician.name AS PhysicianName,
    Physician.position
FROM 
    Physician
WHERE 
    Physician.physicianid IN (
        SELECT physicianid 
        FROM PhysicianProcedure 
        GROUP BY physicianid 
        HAVING COUNT(procedureid) > 1
    )
ORDER BY 
    Physician.physicianid;
""", conn)

,PhysicianName,position
0,Christopher Turk,Surgical Attending Physician
1,John Wen,Surgical Attending Physician


6. Write an SQL to aggregate the results by using multiple columns in the SELECT clause. Interpret the output.

In [34]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName,
    COUNT(Undergoes.procedureid) AS NumberOfProcedures,
    SUM(Procedure.cost) AS TotalCost
FROM 
    Undergoes
INNER JOIN 
    Patient ON Undergoes.patientid = Patient.patientid
INNER JOIN 
    Procedure ON Undergoes.procedureid = Procedure.procedureid
GROUP BY 
    Patient.patientid, Patient.name
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,NumberOfProcedures,TotalCost
0,John Smith,3,9375.0
1,Dennis Doe,3,16399.0


7. Write a subquery using the NOT IN operator. Show the results and sort the results by key field(s). Interpret the output.

In [35]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName,
    Patient.patientid
FROM 
    Patient
WHERE 
    Patient.patientid NOT IN (
        SELECT Undergoes.patientid 
        FROM Undergoes
        INNER JOIN Procedure ON Undergoes.procedureid = Procedure.procedureid
        WHERE Procedure.cost >= (SELECT AVG(cost) FROM Procedure)
    )
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,patientid
0,Grace Ritchie,100000002
1,Random J. Patient,100000003


8. Write a query using a CASE statement. Show the results and sort the results by key field(s). Interpret the output.

In [36]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName,
    Patient.patientid,
    Physician.position AS PhysicianPosition,
    CASE 
        WHEN Physician.position LIKE '%Specialist%' THEN 'Specialist'
        WHEN Physician.position LIKE '%General%' THEN 'General'
        ELSE 'Other'
    END AS PhysicianCategory
FROM 
    Patient
INNER JOIN 
    Physician ON Patient.pcp = Physician.physicianid
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,patientid,PhysicianPosition,PhysicianCategory
0,John Smith,100000001,Staff Internist,Other
1,Grace Ritchie,100000002,Attending Physician,Other
2,Random J. Patient,100000003,Attending Physician,Other
3,Dennis Doe,100000004,Surgical Attending Physician,Other


9. Write a query using the NOT EXISTS operator. Show the results and sort the results by key field(s). Interpret the output.

In [37]:
pd.read_sql("""SELECT 
    Physician.name AS PhysicianName,
    Physician.physicianid
FROM 
    Physician
WHERE NOT EXISTS (
    SELECT 1 
    FROM PhysicianProcedure 
    WHERE PhysicianProcedure.physicianid = Physician.physicianid
)
ORDER BY 
    Physician.physicianid;
""", conn)

,PhysicianName,physicianid
0,John Dorian,1
1,Elliot Reid,2
2,Percival Cox,4
3,Bob Kelso,5
4,Keith Dudemeister,8
5,Molly Clock,9


10. Write a subquery using the NOT NULL operator in the inner query. Show the results and sort the results by key field(s). Interpret the output.

In [38]:
pd.read_sql("""SELECT 
    Patient.name AS PatientName,
    Patient.patientid,
    Physician.name AS PhysicianName
FROM 
    Patient
INNER JOIN 
    Physician ON Patient.pcp = Physician.physicianid
WHERE 
    Patient.patientid IN (
        SELECT patientid 
        FROM Patient 
        WHERE pcp IS NOT NULL
    )
ORDER BY 
    Patient.patientid;
""", conn)

,PatientName,patientid,PhysicianName
0,John Smith,100000001,John Dorian
1,Grace Ritchie,100000002,Elliot Reid
2,Random J. Patient,100000003,Elliot Reid
3,Dennis Doe,100000004,Christopher Turk
